# Synthetic Stream Gauges: 
## Improving Streamflow Predictions in Unmonitored River Segments through Deep Learning

This notebook details the development and validation of an advanced Long Short-Term Memory (LSTM) neural network model designed to generate and use synthetic data to improve river streamflow predictions in unmonitored segments. Through the simulation of watershed networks and the application of an innovative LSTM-based approach, this study addresses key challenges in hydrology, such as the lack of monitoring data, and demonstrates how artificial intelligence can significantly contribute to water resource management.

To address data sparsity in unmonitored river segments, we generated a synthetic data set representing various hydrologic conditions. Using "leaky bucket" models to simulate tributary behavior, this approach allows for a detailed and varied representation of possible flow dynamics. The resulting synthetic data are instrumental in training our LSTM model, providing a solid foundation for learning the complexities of river streamflows and improving prediction capabilities.

We developed a balanced in depth and complexity LSTM network to accurately capture the streamflow and overflow dynamics in each of the tributary buckets of the simulated watershed, considering the combined output streamflow as the "ground truth" data.

# 1. Setup

The first thing to do is setup the notebook environment with all the libraries, declare model global parameters, settings, variables and functions that define the leaky bucket model and the bucket networks we want to represent, as well as define the hyperparameters and structure for the deep learning model. 

Note: in a typical full-scale modeling framework, these would be decalared in a configuration file.

### 1.1 Importing libraries 

We import standard libraries for data management, calculations and plotting. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import os
import pickle

We import machine learning libraries for modeling architectures, training, validation and testing.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
from torch.autograd import Variable 
import sklearn
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import trange, tqdm
print(torch.__version__)

#### 1.1.1 Importing stored data

We import the synthetic data generated for the watershed network, which includes the streamflow data for each of the tributary buckets and the combined output streamflow. Or we can generate the synthetic data on the fly.

In [ ]:
# Save data dictionaries
def save_dictionaries(basin_dict, combined_dict, basin_file='data/basin_network_dictionary.pkl', combined_file='data/combined_network_dictionary.pkl'):
    with open(basin_file, 'wb') as f:
        pickle.dump(basin_dict, f)
    with open(combined_file, 'wb') as f:
        pickle.dump(combined_dict, f)

# Load data dictionaries
def load_dictionaries(basin_file='data/basin_network_dictionary.pkl', combined_file='data/combined_network_dictionary.pkl'):
    with open(basin_file, 'rb') as f:
        basin_dict = pickle.load(f)
    with open(combined_file, 'rb') as f:
        combined_dict = pickle.load(f)
    return basin_dict, combined_dict

generate_new_data = False

### 1.2 Defining the physical bucket model system

**Global variables**

Constants in our bucket model system are:
- g, gravitational acceleration [$m/s^2$]
- time step [$s$]

In [ ]:
g = 9.80665
time_step = 1

**Parameters for the Forcing Process (Precipitation)**

To generate precipitation data for our hydrological system, we employ a simple random process that determines whether it is raining based on the previous state, as well as the total amount of rain during the event.

We classify precipitation into three types: None, Light, and Heavy. Each type has distinct ranges of rainfall probabilities and depths.

In [ ]:
rain_probability_range = {"None": [0.6, 0.7], 
                          "Light": [0.5, 0.8], 
                          "Heavy": [0.2, 0.3]}

rain_depth_range = {"Light": [0, 2], "Heavy": [2, 8]}

**Bucket physical attributes**

The physical leaky bucket attributes include:
- A_bucket, bucket area [$m^2$]
- A_spigot, spigot areas [$m^2$]
- H_bucket, bucket heights [$m$]
- H_spigot, spigot heights [$m$]
- K_infiltration, infiltration rate [$mm/hr$]
- ET_parameter, evapotranspiration parameter [$mm/day$]

We will generate a diversity of leaky buckets by randomy selecting their physical attribute values from the following possible ranges.

In [ ]:
bucket_attributes_range = {"A_bucket": [1.0, 2.0],
                           "A_spigot": [0.1, 0.2],
                           "H_bucket": [5.0, 6.0],
                           "H_spigot": [1.0, 3.0],
                           "K_infiltration": [1e-7, 1e-9],
                           "ET_parameter": [7, 9]
                          }

bucket_attributes_list = list(bucket_attributes_range.keys())

**Channel physical parameters**

This section includes the channel physical parameters. Initially, the transformation parameters are defined

- transform_type: 0 (null transform), 1 (simple shift), 2 (time-based displacement), 3 (simple attenuation), 4 (LagK), 5 (Muskingum-Cunge)
- shift_amount: position number to shift (applies when transform_type == 1)
- time_shift: specified number of seconds to move the streamflow
- attenuation_factor: value for multiplying the streamflow
- lag_amount: number of streamflow positions to delay the output
- param_list: list of parameters to apply Muskingum-Cunge transformation


In [ ]:
transform_params_range = {"transform_type": [0], #"transform_type": [0, 1, 3, 4, 5], #transform_types 1 to 5 are not completely implemented
                          "shift_amount": [0, 100],
                          # "time_shift": [],
                          "attenuation_factor": [0.01, 0.99],
                          "lag_amount": [0, 100]
                         }

transform_params_list = list(transform_params_range.keys())

**Model input and output variables**

The data fluxes in and out of the leaky bucket considered in this stystem and the leaky bucket state, include
- precipitation into the bucket as a model input (precip) 
- the actual and potential loss to evaporation from the bucket as model inputs (et, pet)
- the water flow over the bucket and out of the spigot as a simulated model output(q_overflow, q_spigot)
- and the state of the water head in the bucket as a simluated model output (h_bucket).

We define lists based on these input and output leaking bucket model variables.

n_combined_input is the number of features to use in the LSTM model. 
n_combined_output is the number of features to predict per each bucket.

In [ ]:
n_buckets_per_network = 2

input_vars = ['precip', 'et', 'q_total_output']
input_vars.extend(bucket_attributes_list)
n_input = len(input_vars) 

# For combined bucket network inputs with combined downstream flow data
input_combined_vars = [f"{var}_b{bucket}" for bucket in range(n_buckets_per_network) for var in ['precip', 'et']]
input_combined_vars.append('q_total_output')
for attribute in bucket_attributes_list:
    for bucket in range(n_buckets_per_network):
        input_combined_vars.append(f"{attribute}_b{bucket}")
output_combined_vars = [f"{var}_b{bucket}" for bucket in range(n_buckets_per_network) for var in ['q_overflow', 'q_spigot']]
n_combined_input = len(input_combined_vars)
n_combined_output = len(output_combined_vars)

print("input_combined_vars: " + str(input_combined_vars))
print("n_combined_input: " + str(n_combined_input))
print("output_combined_vars: " + str(output_combined_vars))
print("n_combined_output: " + str(n_combined_output))


**Data noise**

Because real world systems are noisy, we can add noise to the synthetic data by multiplying the values by a random factor taken from a normal distribution with a mean of 1 and a standard deviation prescribed for different variables.

In [ ]:
is_noise = True
noise = {"pet": 0.1, "et": 0.1, "q": 0.1, "head": 0.1} 

**Illustration of leaking bucket hydrological system**

![Illustration of leaking bucket hydrological system](figs/Leaky_Bucket_Model.png)

### 1.3 Defining the modeling setup 

**Deep Learning Model (LSTM) Hyperparameters**

The hyperparameters for the LSTM (Long Short-Term Memory) deep learning model include:
- device: The device (CPU or CUDA or MPS) used for training and inference.
- hidden_state_size: The number of units in the hidden state of the LSTM model.
- num_layers: Number of LSTM layers.
- num_epochs: Number of training epochs.
- batch_size: Number of samples in each training batch.
- seq_length: Length of input sequences.
- learning_rate: Learning rate for the optimizer.
- num_classes: Number of output classes.
- input_size: Size of the input layer.

These hyperparameters control the behavior and performance of the LSTM model and can be adjusted to optimize the model's accuracy and generalization capabilities for specific tasks and datasets.

In [ ]:
# Automatic selection of device: MPS for Apple Silicon, CUDA for NVIDIA GPUs if available, otherwise CPU
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

print("Using device: ", device)

# Specific configuration for MPS
if device.type == 'mps':
    os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
    print("MPS fallback enabled:", os.environ.get('PYTORCH_ENABLE_MPS_FALLBACK'))

# Model parameters
batch_size = 256 
seq_length = 24
num_epochs_combined = 30
num_layers_combined = 60
hidden_state_size_combined = 36
learning_rate_combined = np.linspace(start=0.1, stop=0.001, num=num_epochs_combined)


**Size of data records and splits**

The deep learning framework requires the data to be split into 3 different sets
- Back propogation is done on the training set (train)
- Hyperparameter tuning is done on the validation set (val)
- Calculating model accuracy is done on the testing set (test)

We will define how many randomly generated basin network configurations will be used for the "train", "val" and "test" sets, as well as the length of the simulations for each set.

Additionally it is necessary to define the number of buckets in every basin network (n_bucket_per_network). Consequently, the number of buckets is equal to the number of basin networks multiplied by the number of buckets per network.

In [ ]:
time_splits = {"train": 7000, "val": 1500,"test": 1500}
n_basin_networks_split = {"train": 350, "val": 204, "test": 204}
n_buckets_split = {"train": n_basin_networks_split["train"] * n_buckets_per_network, "val": n_basin_networks_split["val"] * n_buckets_per_network,"test": n_basin_networks_split["test"] * n_buckets_per_network}

We use the settings above to calculate the total length of the data record to generate and the total number of buckets and basin networks

In [ ]:
num_records = time_splits["train"] + time_splits["val"] + time_splits["test"] + seq_length * 3
n_buckets = n_buckets_split["train"] + n_buckets_split["val"] + n_buckets_split["test"]
n_basin_networks = n_basin_networks_split["train"] + n_basin_networks_split["val"] + n_basin_networks_split["test"]
print(n_basin_networks)

We then calculate the parameters for the bucket and time splits necessary to feed the model with separate datasets for training, validation, and testing.

We need to create a new function to calculate parameter for the basin network splits into separate datasets.

In [ ]:
def split_parameters():
    # create lists of bucket indices for each set based on the given bucket splits
    buckets_for_training = list(range(0, n_buckets_split['train'] + 1))
    buckets_for_val = list(range(n_buckets_split['train'] + 1, 
                                 n_buckets_split['train'] + n_buckets_split['val'] + 1))
    buckets_for_test = list(range(n_buckets - n_buckets_split['test'], n_buckets))

    # determine the time range for each set based on the given time splits
    train_start = seq_length
    train_end   = time_splits["train"]
    val_start   = train_end + seq_length
    val_end     = val_start + time_splits["val"]
    test_start  = val_end + seq_length
    test_end    = test_start + time_splits["test"]
    
    # organize the split parameters into separate lists for each set
    train_split_parameters = [buckets_for_training, train_start, train_end]
    val_split_parameters = [buckets_for_val, val_start, val_end]
    test_split_parameters = [buckets_for_test, test_start, test_end]
    
    return [train_split_parameters, val_split_parameters, test_split_parameters]

In [ ]:
[[buckets_for_training, train_start, train_end],
[buckets_for_val, val_start, val_end],
[buckets_for_test, test_start, test_end]]= split_parameters()
print([[buckets_for_training, train_start, train_end],
[buckets_for_val, val_start, val_end],
[buckets_for_test, test_start, test_end]])

This function will create separate datasets for training, validation, and testing.

In [ ]:
def basin_split_param():
    #create lists of basin indices for each set
    basins_train = list(range(0, n_basin_networks_split['train']))
    basins_val = list(range(n_basin_networks_split['train'], 
                            n_basin_networks_split['train'] + n_basin_networks_split['val']))
    basins_test = list(range(n_basin_networks - n_basin_networks_split['test'], n_basin_networks)) 
    
    #place basin split parameters into separate lists
    train_basin_split_param = [basins_train]
    val_basin_split_param = [basins_val]
    test_basin_split_param = [basins_test]
    
    #determine transform parameters per basin network
    
    
    return [train_basin_split_param, val_basin_split_param, test_basin_split_param]

This will run the function above and print out the range for the whole basin network split

In [ ]:
[[basin_networks_train],[basin_networks_val],[basin_networks_test]]= basin_split_param()
print([[basin_networks_train],[basin_networks_val],[basin_networks_test]])

### 1.4 Creating a sample of diverse buckets

Now that we know how many buckets we need, we can generate a sample of diverse buckets with their respective boundary and initial conditions by randomly sampling from the possible ranges for each attribute defined in the settings above.

In [ ]:
def setup_buckets(n_buckets):
    # Boundary conditions
    buckets = {bucket_attribute:[] for bucket_attribute in bucket_attributes_list}
    for i in range(n_buckets):
        for attribute in bucket_attributes_list:
            buckets[attribute].append(np.random.uniform(bucket_attributes_range[attribute][0], 
                                                        bucket_attributes_range[attribute][1]))

    # Initial conditions
    h_water_level = [np.random.random() for i in range(n_buckets)]
    mass_overflow = [np.random.random() for i in range(n_buckets)]
    return buckets, h_water_level, mass_overflow

In [ ]:
buckets, h_water_level, mass_overflow = setup_buckets(n_buckets)

### 1.5 Creating the synthetic "precipitation" 

We first randomly assign rainfall parameters for each bucket, using the probability ranges specified for each precipitation type. We then generate synthetic input time series for each bucket model, known as forcing data. This process involves using the previously defined rainfall parameters and the random process defined in the function below.

In [ ]:
def pick_rain_params():
    buck_rain_params = [rain_depth_range,
                        np.random.uniform(rain_probability_range["None"][0],
                                            rain_probability_range["None"][1]),
                        np.random.uniform(rain_probability_range["Heavy"][0],
                                            rain_probability_range["Heavy"][1]),
                        np.random.uniform(rain_probability_range["Light"][0],
                                            rain_probability_range["Light"][1])
                 ]
    return buck_rain_params

In [ ]:
def random_rain(preceding_rain, bucket_rain_params):
    depth_range, no_rain_probability, light_rain_probability, heavy_rain_probability = bucket_rain_params
    # some percent of time we have no rain at all
    if np.random.uniform(0.01, 0.99) < no_rain_probability:
        rain = 0

    # When we do have rain, the probability of heavy or light rain depends on the previous day's rainfall
    else:
        # If yesterday was a light rainy day, or no rain, then we are likely to have light rain today
        if preceding_rain < depth_range["Light"][1]:
            if np.random.uniform(0, 1) < light_rain_probability:
                rain = np.random.uniform(0, 1)
            else:
                # But if we do have heavy rain, then it could be very heavy
                rain = np.random.uniform(depth_range["Heavy"][0], depth_range["Heavy"][1])

        # If it was heavy rain yesterday, then we might have heavy rain again today
        else:
            if np.random.uniform(0, 1) < heavy_rain_probability:
                rain = np.random.uniform(0, 1)
            else:
                rain = np.random.uniform(depth_range["Light"][0], depth_range["Light"][1])
    return rain

We generate a random rainfall input timeseries for each bucket and store it in a dictionary

In [ ]:
in_list = {}
for ibuc in range(n_buckets):
    bucket_rain_params = pick_rain_params()
    in_list[ibuc] = [0]
    for i in range(1, num_records):
        in_list[ibuc].append(random_rain(in_list[ibuc][i-1], bucket_rain_params))

### 1.6 Running Numerical Simulations of the Bucket Model to Generate "Ground Truth" Data

We run bucket model simulations to generate the data for our system. 

We perform numerical simulations of a bucket model for a specific bucket index. It iterates over each time step, updating the water level based on precipitation, evapotranspiration, infiltration, overflow, and spigot outflow. The simulation results, including the water level, overflow, spigot flow, and other attributes, are stored in a data frame. This is a concise representation of the bucket model's behavior over time for the given bucket index.

In [ ]:
def run_bucket_simulation(ibuc, noise):
    columns = ['precip', 'et', 'h_bucket', 'q_overflow', 'q_spigot', 'q_total']
    columns.extend(bucket_attributes_list)
    
    # Precompute g (acceleration due to gravity)
    # g = 9.81

    # Pre-generate random numbers for noise
    if is_noise:
        pet_noise = np.random.normal(1, noise['pet'], len(in_list[ibuc]))
        et_noise = np.random.normal(1, noise['et'], len(in_list[ibuc]))
        head_noise = np.random.normal(1, noise['head'], len(in_list[ibuc]))
        q_noise = np.random.uniform(0, noise['q'], len(in_list[ibuc]))

    # Memory to store model results
    data = []

    # Main loop through time
    for t, precip_in in enumerate(in_list[ibuc]):
        # Add the input mass to the bucket
        h_water_level[ibuc] = h_water_level[ibuc] + precip_in

        # Lose mass out of the bucket (evaporation and infiltration)
        et = np.max([0, (buckets["A_bucket"][ibuc] / buckets["ET_parameter"][ibuc]) * np.sin(t) * pet_noise[t]])
        infiltration = h_water_level[ibuc] * buckets["K_infiltration"][ibuc]
        h_water_level[ibuc] = np.max([0, h_water_level[ibuc] - et])
        h_water_level[ibuc] = np.max([0, h_water_level[ibuc] - infiltration])
        if is_noise:
            h_water_level[ibuc] *= et_noise[t]

        # Overflow if the bucket is too full
        if h_water_level[ibuc] > buckets["H_bucket"][ibuc]:
            mass_overflow[ibuc] = h_water_level[ibuc] - buckets["H_bucket"][ibuc]
            h_water_level[ibuc] = buckets["H_bucket"][ibuc]
            if is_noise:
                h_water_level[ibuc] -= q_noise[t]

        # Calculate head on the spigot
        h_head_over_spigot = h_water_level[ibuc] - buckets["H_spigot"][ibuc]
        if is_noise:
            h_head_over_spigot *= head_noise[t]

        # Calculate water leaving the bucket through the spigot
        if h_head_over_spigot > 0:
            velocity_out = np.sqrt(2 * g * h_head_over_spigot)
            spigot_out = velocity_out * buckets["A_spigot"][ibuc] * time_step
            h_water_level[ibuc] -= spigot_out
        else:
            spigot_out = 0

        # Save the data in time series
        data.append([precip_in, et, h_water_level[ibuc], mass_overflow[ibuc], spigot_out, mass_overflow[ibuc] + spigot_out] +
                    [buckets[attribute][ibuc] for attribute in bucket_attributes_list])

        mass_overflow[ibuc] = 0

    # Convert data to DataFrame
    df = pd.DataFrame(data, columns=columns)
    return df


Then, we run and store the simulations for each bucket in a dictionnary

We ran basin network model simulation to create a dictionary that contains n_basin_networks elements. Each element represents a network of n_buckets_per_network buckets.

In [ ]:
def run_basin_network_simultation():
    result = {}
    result['buckets'] = {}
    for i in range(n_basin_networks):
        bucket_dictionary = {}
        for j in range(n_buckets_per_network):
            bucket_dictionary[j] = run_bucket_simulation((i * n_buckets_per_network) + j, noise)
        result['buckets'][i] = bucket_dictionary
    return result

In [ ]:
basin_network_dictionary = {}

if generate_new_data or not (os.path.exists('data/basin_network_dictionary.pkl') and os.path.exists('data/combined_network_dictionary.pkl')):
    basin_network_dictionary = run_basin_network_simultation()
else:
    basin_network_dictionary, _ = load_dictionaries()

### 1.7 Create a combined streamflow

We create a function to transform streamflow. 

The data in each bucket network and these combined streamflow represent the "ground truth", which is what we will try to learn with the LSTM. 

The transformation type to perform can be one of the following:

- **Null transformation:** Returns the same series of input streamflows
- **Simple shift:** Move the elements right or left according to parameters shift_amount
- **Time-based shift:** Move the elements to the right or left according to the specified number of seconds time_shift; requires some information about the internal time values of the series.
- **Simple Attenuation:** multiply each element of the series by an attenuation factor
- **LagK:** Move each element to position K (lag_amount)
- **Muskingum-Cunge:** Muskingum-Cunge method to combine river flows

In [ ]:
#def mcTransform(Q, param_list):
#    return Q

def transformQ(Q, T):
    # Verify if the transformation parameters are null
    if T[0] == 0 or T[0] is None:
        # The null transformation returns the same series of input streamflows
        return Q
    else:
        transform_type = T[0]
        if transform_type == 1:
            # Implement the simple shift transformation
            # Move the elements to the right or left according to parameters
            shift_amount = T[1]
            #print(shift_amount)
            #print(Q[shift_amount:])
            #print(Q[:shift_amount])
            #transformed_Q = Q[shift_amount:] + Q[:shift_amount]
            transformed_Q = np.roll(Q, shift_amount).tolist()
        elif transform_type == 2:
            # Implementing time-based displacement transformation
            # Move the elements to the right or left according to the specified number of seconds
            time_shift = T[1]
            # Information about the internal time values of the series should be used here
            # to calculate the appropriate displacement based on the specified seconds
            # transformed_Q = ...
            # Implement the code for this transformation according to needs and available data
        elif transform_type == 3:
            # Implement the simple attenuation transformation
            # Multiplying each element of the series by an attenuation factor
            attenuation_factor = T[1]
            transformed_Q = [q * attenuation_factor for q in Q]
        elif transform_type == 4:
            # Implement LagK transformation
            # Move each element to position K
            lag_amount = T[1]
            transformed_Q = [Q[i - lag_amount] if i >= lag_amount else 0 for i in range(len(Q))]
        elif transform_type == 5:
            # Implement the Muskingum-Cunge method to combine river flows
            param_list = T[1]
            # transformed_Q = mcTransform(Q, param_list)
            # Implement the code for this transformation according to needs and available data
        #else:
            # In case new transformations are added in the future, they can be handled here
            # by adding more elif blocks
        return transformed_Q


Now we combine the transformed streamflows of a basin network in one single streamflow

In [ ]:
def get_param_key(transform_type):
    if transform_type == 0:
        return None
    elif transform_type == 1:
        return "shift_amount"
    #elif transform_type == 2:  # For time-based transformation (not implemented yet)
    #    return "time_shift"
    elif transform_type == 3:
        return "attenuation_factor"
    elif transform_type == 4:
        return "lag_amount"
    #elif tranform_type == 5:  # For Muskingum-Cunge transformation (not implemented yet)
    #    return "param_list"

In [ ]:
def generate_transform_params(bucket_basin):
    n = len(bucket_basin)
    transform_params = []
    for i in range(n - 1):
        transform_type = np.random.choice(transform_params_range["transform_type"])
        param_key = get_param_key(transform_type)
        if transform_type == 0:
            param_value = None
        else:
            param_range = transform_params_range[param_key]
            #param_value = None
            if param_key in ["shift_amount", "lag_amount"]:
                param_value = np.random.randint(param_range[0], param_range[1] + 1)
            else:
                param_value = np.random.uniform(param_range[0], param_range[1])
        transform_params.append((transform_type, param_value))
    transform_params.append((0, None))
    return transform_params

In [ ]:
def generate_transform_params_network(basin_network_dictionary):
    transform_params_vector = []
    for i in range(n_basin_networks):
        transform_params_vector.append([i, generate_transform_params(basin_network_dictionary['buckets'][i])])
    return transform_params_vector

In [ ]:
transform_params_vector = generate_transform_params_network(basin_network_dictionary)

In [ ]:
def combine_streamflows(flows, transform_params):
    transformed_flows = []
    
    for Q, T in zip(flows, transform_params):
        transformed_flows.append(transformQ(Q, T))

    return np.sum(transformed_flows, axis=0)

In [ ]:

def streamflows_output(transform_params_vector):
    combined_streamflows_dictionary = {} 
    q_total_matrix = []
    for i in range(n_basin_networks):
        q_total_vector = []
        for j in range(n_buckets_per_network):
            q_total_vector.append(basin_network_dictionary['buckets'][i][j]['q_total'])
        q_total_matrix.append(combine_streamflows(q_total_vector, transform_params_vector[i][1]))
    combined_streamflows_dictionary['q_total_output'] = q_total_matrix
    return combined_streamflows_dictionary


In [ ]:
combined_streamflows_dictionary = streamflows_output(transform_params_vector)

In [ ]:
print(len(combined_streamflows_dictionary['q_total_output']))

In [ ]:
for inetwork in range(n_basin_networks):
    q_total_output_values = combined_streamflows_dictionary['q_total_output'][inetwork]
    
    for ibuc in range(n_buckets_per_network):
        bucket_df = basin_network_dictionary['buckets'][inetwork][ibuc]
        bucket_df['q_total_output'] = q_total_output_values


In [ ]:
print(basin_network_dictionary['buckets'][0][0].keys())
print(basin_network_dictionary)

In [ ]:
# Create combined network dictionary
combined_network_dictionary = {'networks': {}}

if generate_new_data or not (os.path.exists('data/basin_network_dictionary.pkl') and os.path.exists('data/combined_network_dictionary.pkl')):

    for network_id in basin_network_dictionary['buckets'].keys():
        combined_df_list = [] 
        q_total_output_series = None  

        for bucket_id in range(n_buckets_per_network):
            df = basin_network_dictionary['buckets'][network_id][bucket_id]
            if q_total_output_series is None:
                q_total_output_series = df['q_total_output']
            df = df.rename(columns={col: f"{col}_b{bucket_id}" for col in df.columns if col != 'q_total_output'})
            df = df.drop(columns=['q_total_output'])
            combined_df_list.append(df)

        combined_df = pd.concat(combined_df_list, axis=1)
        combined_df['q_total_output'] = q_total_output_series
        combined_network_dictionary['networks'][network_id] = combined_df
else:
    _, combined_network_dictionary = load_dictionaries()

In [ ]:
if generate_new_data or not (os.path.exists('data/basin_network_dictionary.pkl') and os.path.exists('data/combined_network_dictionary.pkl')):
    save_dictionaries(basin_network_dictionary, combined_network_dictionary)


In [ ]:
print(combined_network_dictionary['networks'][0].keys())
print(combined_network_dictionary)

This function retrieves data from a specific bucket within a given basin network. It takes three arguments:
- **inetwork**: The index of the watershed network from which data will be retrieved.
- **ibuc**: The index of the specific bucket within the basin network.
- **basin_network_dictionary**: A dictionary containing all the basin networks and their respective buckets.

The function returns a DataFrame **df** containing the data of the specified bucket.

In [ ]:
def retrieve_data(inetwork, ibuc, basin_network_dictionary):
    df = basin_network_dictionary['buckets'][inetwork][ibuc]
    return df

def retrieve_combined_data(inetwork, combined_network_dictionary):
    df = combined_network_dictionary['networks'][inetwork]
    return df

### 1.8 Visualizing a sample of the bucket fluxes
We plot the model simulations to ensure the existence of fluxes and validate that the generated values are realistics for both the spigot (channel flow) and over the top (flooding). 

In [ ]:
def viz_simulation(basin_network_dictionary, inetwork, ibuc):
    fig = plt.figure(figsize=(12, 3))
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)
    
    print('Bucket:', ibuc)
    print("Overflow mean:", np.round(basin_network_dictionary['buckets'][inetwork][ibuc].q_overflow.mean(),2))
    print("Overflow max:", np.round(basin_network_dictionary['buckets'][inetwork][ibuc].q_overflow.max(),2))

    input_vars_for_plot = [var for var in input_vars if var != 'q_total_output']
    
    basin_network_dictionary['buckets'][inetwork][ibuc].loc[:100, input_vars_for_plot].plot(ax=ax1)
    basin_network_dictionary['buckets'][inetwork][ibuc].loc[:100, ['q_overflow', 'q_spigot','h_bucket']].plot(ax=ax2)

    ax1.set_title('Model inputs')
    ax1.set_xlabel('Time (hours)')
    ax1.set_ylabel('Values')
    ax2.set_title('Model outputs')
    ax2.set_xlabel('Time (hours)')
    ax2.set_ylabel('Flow/Level')
    plt.legend()
    plt.show()
    plt.close()



In [ ]:
for ibuc in range(n_buckets_per_network):
    viz_simulation(basin_network_dictionary, n_basin_networks_split['train'], ibuc)

We also plot the combined flow of a network to verify the consistency of the transformation function.

In [ ]:
def viz_network(basin_network_dictionary, inetwork):
    fig = plt.figure(figsize=(12, 6))

    for ibuc in range(n_buckets_per_network):
        print('Bucket:', ibuc)
        print("Total mean:", np.round(basin_network_dictionary['buckets'][inetwork][ibuc].q_total.mean(),2))
        print("Total max:", np.round(basin_network_dictionary['buckets'][inetwork][ibuc].q_total.max(),2))

        plt.plot(basin_network_dictionary['buckets'][inetwork][ibuc].loc[:100,"q_total"], label='Bucket '+str(ibuc)+' q_total')
        
    q_total_output = basin_network_dictionary['buckets'][inetwork][0].loc[:100,"q_total_output"]

    plt.plot(q_total_output, color='k', linestyle='dashed', label='q_total_output')

    plt.title('Combined streamflows per Network ' + str(inetwork) + ': q_total = q_spigot + q_overflow')
    plt.xlabel('Time (hours)')
    plt.ylabel('Flow rate (m$^3$/s)')
    plt.legend()
    plt.show()
    plt.close()


In [ ]:
countN = 0
for inetwork in basin_networks_val:
    countN += 1
    viz_network(basin_network_dictionary, inetwork)
    if countN == 10:
        break

# 2. Deep learning model
This section sets up our deep learning model and training procedure.

### 2.1 Defining the neural network model
This is the part of the notebook where we will be using the previous simulations to learn from the generated data and subsequently make predictions. Here, we leverage the simulations to extract valuable insights and apply them in our learning and prediction processes.

Here we define a class called LSTM1, which is a PyTorch module for a multi-layer Long Short-Term Memory (LSTM) network. 

**Architecture**

![Illustration of LSTM Network Architecture](figs/LSTM_Architecture.png)

**Brief explanation**
- The input to the module is a tensor x of shape (`batch_size`, `seq_length`, `input_size`), which represents a sequence of batch_size samples, each of length seq_length, with input_size features at each time step. 
- The LSTM layer is defined using the nn.LSTM class, with input_size as the size of the input layer, hidden_size as the size of the hidden state, and batch_first=True indicating that the first dimension of the input tensor is the batch size. 
- The output of the LSTM layer is passed through a ReLU activation function, and then to a fully connected layer (nn.Linear) with num_classes output units. 
- The forward method takes the input tensor x as an argument, along with an optional tuple `init_states` representing the initial hidden and internal states of the LSTM layer, and returns the output tensor prediction. 
- If `init_states` is not provided, it is initialized as a tensor of zeros with shape (`batch_size`, `hidden_size`).

In [ ]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, batch_size, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length 
        self.batch_size = batch_size

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.fc_1 =  nn.Linear(hidden_size, num_classes) #fully connected 1
   
    def forward(self, x, init_states=None):

        if init_states is None:
            h_t = Variable(torch.zeros(1, x.size(0), self.hidden_size, device=device)) # hidden state
            c_t = Variable(torch.zeros(1, x.size(0), self.hidden_size, device=device)) # internal state
            init_states = (h_t, c_t)
           
        out, _ = self.lstm(x, init_states)
        out = self.relu(out)
        prediction = self.fc_1(out) # Dense, fully connected layer
        
        return prediction

### 2.2 Defining a procedure for model validation
Here we verify that our model is working the way we expect. We would especially want to check model validation when changing hyperparameters.

We define a function that validates and tests the LSTM model, as well as checks the water balance of the system. 

**Brief explanation**
- We use the pre-defined LSTM model to make predictions on the validation data. 
- The output of this model is then used to compute four different metrics, the Nash-Sutcliffe Efficiency (NSE), Root Mean Sqaure Error (RMSE), Mean Absolute Error (MAE) for the spigot_out, and  mass_overflow columns of the dataframe.
- We plot the actual spigot_out and mass_overflow values against their corresponding LSTM predictions. 
- We check the water balance of the system by summing up the input, evapotranspiration, mass_overflow, spigot_out, and the last recorded water level in the dataframe, and compare this to the total mass out of or left in the system. 
- We print out the percent mass residual as a measure of how well the system is balanced.

In [ ]:
def check_validation_period_combined(lstm_combined, combined_np_val_seq_X, combined_network_dictionary, inetwork, n_plot=100, verbose=False):
    def __make_prediction(df, ibuc):
        lstm_output_val = lstm_combined(torch.Tensor(combined_np_val_seq_X[inetwork]).to(device=device))
        val_spigot_prediction = []
        val_overflow_prediction = []

        for i in range(lstm_output_val.shape[0]):
            spigot_pred = (lstm_output_val[i, -1, (ibuc * 2) + 1].cpu().detach().numpy() * np.std(df.loc[train_start:train_end, 'q_spigot_b' + str(ibuc)])) + np.mean(df.loc[train_start:train_end, 'q_spigot_b' + str(ibuc)])
            overflow_pred = (lstm_output_val[i, -1, (ibuc * 2)].cpu().detach().numpy() * np.std(df.loc[train_start:train_end, 'q_overflow_b' + str(ibuc)])) + np.mean(df.loc[train_start:train_end, 'q_overflow_b' + str(ibuc)])

            #spigot_pred = (lstm_output_val[i, -1, (ibuc * 2) + 1].cpu().detach().numpy() * spigot_std) + spigot_mean
            #overflow_pred = (lstm_output_val[i, -1, (ibuc * 2)].cpu().detach().numpy() * overflow_std) + overflow_mean

            spigot_pred = max(spigot_pred, 0)
            overflow_pred = max(overflow_pred, 0)

            val_spigot_prediction.append(spigot_pred)
            val_overflow_prediction.append(overflow_pred)

        return val_spigot_prediction, val_overflow_prediction

    def __compute_metrics_combined(df, val_spigot_prediction, val_overflow_prediction, ibuc):
        # Use specific columns for each bucket
        spigot_out = df.loc[val_start:val_end, 'q_spigot_b' + str(ibuc)]
        spigot_mean = np.mean(spigot_out)
        spigot_pred_variance = 0
        spigot_obs_variance = 0
        spigot_abs_diff = 0

        overflow_out = df.loc[val_start:val_end, 'q_overflow_b' + str(ibuc)]
        overflow_mean = np.mean(overflow_out)
        overflow_pred_variance = 0
        overflow_obs_variance = 0
        overflow_abs_diff = 0

        for i, pred_spigot in enumerate(val_spigot_prediction):
            t = i + seq_length - 1
            spigot_pred_variance += np.power((pred_spigot - spigot_out.iloc[t]), 2)
            spigot_obs_variance += np.power((spigot_mean - spigot_out.iloc[t]), 2)
            spigot_abs_diff += np.abs(pred_spigot - spigot_out.iloc[t])

        for i, pred_overflow in enumerate(val_overflow_prediction):
            t = i + seq_length - 1
            overflow_pred_variance += np.power((pred_overflow - overflow_out.iloc[t]), 2)
            overflow_obs_variance += np.power((overflow_mean - overflow_out.iloc[t]), 2)            
            overflow_abs_diff += np.abs(pred_overflow - overflow_out.iloc[t])

        spigot_nse = 1 - (spigot_pred_variance / spigot_obs_variance)
        overflow_nse = 1 - (overflow_pred_variance / overflow_obs_variance)
        
        spigot_rmse = np.sqrt(spigot_pred_variance / len(val_spigot_prediction))
        overflow_rmse = np.sqrt(overflow_pred_variance / len(val_overflow_prediction))
        
        spigot_mae = spigot_abs_diff / len(val_spigot_prediction)
        overflow_mae = overflow_abs_diff / len(val_overflow_prediction)

        return spigot_nse, overflow_nse, spigot_rmse, overflow_rmse, spigot_mae, overflow_mae

    def __compute_mass_balance_combined(df, ibuc):
        suffix = '_b' + str(ibuc)  # Suffix to identify the specific bucket
        mass_in = df.sum()['precip' + suffix]
        mass_out = df.sum()['et' + suffix] + df.sum()['q_overflow' + suffix] + df.sum()['q_spigot' + suffix] + df['h_bucket' + suffix].iloc[-1]
        return mass_in, mass_out

    if verbose: print("*** Network", inetwork, "***")

    df = retrieve_combined_data(inetwork, combined_network_dictionary)  # Retrieves the combined DataFrame for this network
    metrics_list = []

    for ibuc in range(n_buckets_per_network):
        # Prediction is made with the combined DataFrame, but specifying the bucket
        val_spigot_prediction, val_overflow_prediction = __make_prediction(df, ibuc)
        # Compute the metrics for the predictions of this bucket
        spigot_nse, overflow_nse, spigot_rmse, overflow_rmse, spigot_mae, overflow_mae = __compute_metrics_combined(df, val_spigot_prediction, val_overflow_prediction, ibuc)
        mass_in, mass_out = __compute_mass_balance_combined(df, ibuc)

        new_row = {
            'spigot_nse': spigot_nse, 'overflow_nse': overflow_nse, 
            'spigot_rmse': spigot_rmse, 'overflow_rmse': overflow_rmse, 
            'spigot_mae': spigot_mae, 'overflow_mae': overflow_mae,
            'mass_in': mass_in, 'mass_out': mass_out
        }
        metrics_list.append(new_row)

        if verbose:

            print("Bucket", ibuc)
            print("NSE Spigot: ", spigot_nse)
            print("NSE Overflow: ", overflow_nse)
            print("RMSE Spigot: ", spigot_rmse)
            print("RMSE Overflow: ", overflow_rmse)
            print("MAE Spigot: ", spigot_mae)
            print("MAE Overflow: ", overflow_mae)
            print("Mass into the system: ", mass_in)
            print("Mass out or left over:", mass_out)
            print("Percent mass residual: {:.0%}".format((mass_in - mass_out) / mass_in))

            # Plot the predictions for this bucket
            fig = plt.figure(figsize=(12, 3))
            ax1 = fig.add_subplot(1, 2, 1)
            ax2 = fig.add_subplot(1, 2, 2)
            ax1.plot(df.loc[val_start + seq_length - 1:val_start + n_plot + seq_length - 1, 'q_spigot_b' + str(ibuc)].values, label = "Spigot out")
            ax1.plot(val_spigot_prediction[:n_plot], label="LSTM spigot out")
            ax1.legend(loc="upper right")
            ax1.set_xlabel('Time (hours)')
            ax1.set_ylabel('Flow rate (m$^3$/s)')
            ax2.plot(df.loc[val_start + seq_length - 1:val_start + n_plot + seq_length - 1, 'q_overflow_b' + str(ibuc)].values, label = "Overflow")
            ax2.plot(val_overflow_prediction[:n_plot], label="LSTM Overflow")
            ax2.legend(loc="upper right")
            ax2.set_xlabel('Time (hours)')
            ax2.set_ylabel('Flow rate (m$^3$/s)')
            plt.show()
            plt.close()

    metrics_df = pd.DataFrame(metrics_list)
    avg_metrics = metrics_df[['spigot_nse', 'overflow_nse', 'spigot_rmse', 'overflow_rmse', 'spigot_mae', 'overflow_mae']].mean()
    total_mass_in = metrics_df['mass_in'].sum()
    total_mass_out = metrics_df['mass_out'].sum()
    total_mass_residual_percent = (total_mass_in - total_mass_out) / total_mass_in

    #network_metrics_df = avg_metrics.to_frame().transpose()
    network_metrics_df = pd.DataFrame([avg_metrics])
    network_metrics_df['total_mass_in'] = total_mass_in
    network_metrics_df['total_mass_out'] = total_mass_out
    network_metrics_df['total_mass_residual_percent'] = total_mass_residual_percent

    if verbose:
        print("*** Network Metrics ***")
        print("Network", inetwork)
        print("NSE Spigot: ", network_metrics_df['spigot_nse'].iloc[0])
        print("NSE Overflow: ", network_metrics_df['overflow_nse'].iloc[0])
        print("RMSE Spigot: ", network_metrics_df['spigot_rmse'].iloc[0])
        print("RMSE Overflow: ", network_metrics_df['overflow_rmse'].iloc[0])
        print("MAE Spigot: ", network_metrics_df['spigot_mae'].iloc[0])
        print("MAE Overflow: ", network_metrics_df['overflow_mae'].iloc[0])
        print("Mass into the system: ", network_metrics_df['total_mass_in'].iloc[0])
        print("Mass out or left over: ", network_metrics_df['total_mass_out'].iloc[0])
        print("Percent mass residual: {:.0%}".format(network_metrics_df['total_mass_residual_percent'].iloc[0]))

    return network_metrics_df


### 2.3 Instantiating the neural network model (LSTM)

Using the hyperparameters from Section 1.2, we define a specific instance of the LSTM model and set up the LSTM

In [ ]:
## For combined bucket network inputs with combined downstream flow data
torch.manual_seed(1)
lstm_combined = LSTM1(num_classes=n_combined_output,  
                        input_size=n_combined_input,    
                        hidden_size=hidden_state_size_combined, 
                        num_layers=num_layers_combined, 
                        batch_size=batch_size, 
                        seq_length=seq_length).to(device=device)

In [ ]:
print(n_combined_input, n_combined_output, seq_length)

### 2.4 Setting up the data to feed into the model

We will set up data for:
- training, to calculate the loss which is backpropogated through the model
- validation, where we get predictions from the trained model and see if the performance is up to our standards
- testing, the data we will utimately use to report the LSTM performance. 

Note: testing is the last thing we would do, if we go back to validation after this step, we would be P-hacking.

**Fitting a scaler to the training set to transform all the data**

Here we fit a scaler to the training set, allowing for the transformation of input and output variables to a normalized and standardized scale, which helps in training the model and maintaining consistency across different datasets. This normalization step ensures that the data is suitable for training our LSTM model. 

In [ ]:
def fit_combined_scaler(combined_network_dictionary):
    # Adjust scaler for combined input
    frames_in = [combined_network_dictionary['networks'][inetwork].loc[train_start:train_end, input_combined_vars] for inetwork in basin_networks_train]
    df_in_combined = pd.concat(frames_in)
    scaler_in_combined = StandardScaler()
    scaler_train_in_combined = scaler_in_combined.fit_transform(df_in_combined)
    # Adjust scaler for combined output
    frames_out = [combined_network_dictionary['networks'][inetwork].loc[train_start:train_end, output_combined_vars] for inetwork in basin_networks_train]
    df_out_combined = pd.concat(frames_out)
    scaler_out_combined = StandardScaler()
    scaler_train_out_combined = scaler_out_combined.fit_transform(df_out_combined)

    return scaler_in_combined, scaler_out_combined



In [ ]:
## For combined bucket network inputs with combined downstream flow data
scaler_in_combined, scaler_out_combined = fit_combined_scaler(combined_network_dictionary)

**Function to create data loader for each data split**

We prepare and organize the data for training. We create data loaders that handle batch processing and shuffling of the data. We also add some preprocessing steps such as scaling the input and output variables. The data loaders and numpy arrays are used for feeding the data into the neural network during the training process.

In [ ]:
def make_combined_data_loader(start, end, basin_networks_list, combined_network_dictionary, input_combined_vars, output_combined_vars, scaler_in_combined, scaler_out_combined, n_combined_input):
    loader = {}
    np_seq_X = {}
    np_seq_y = {}
    
    # Traverse the networks in the provided list
    for inetwork in basin_networks_list:
        # Assuming each dataframe in the network covers the same time range
        df_combined = combined_network_dictionary['networks'][inetwork]
        
        # Transform the whole dataframe for inputs
        scaler_in_i = scaler_in_combined.transform(df_combined.loc[start:end, input_combined_vars])
        np_seq_X_ibuc = np.zeros((scaler_in_i.shape[0] - seq_length, seq_length, n_combined_input))
        
        for i in range(scaler_in_i.shape[0] - seq_length):
            t = i + seq_length
            np_seq_X_ibuc[i, :, :] = scaler_in_i[i:t, :]

        # Prepare the output data
        scaler_out_i = scaler_out_combined.transform(df_combined.loc[start:end, output_combined_vars])
        np_seq_y_ibuc = np.zeros((scaler_out_i.shape[0] - seq_length, seq_length, n_combined_output))
            
        for i in range(scaler_out_i.shape[0] - seq_length):
            t = i + seq_length
            np_seq_y_ibuc[i, :, :] = scaler_out_i[i:t, :]
            
        np_seq_y[inetwork] = np_seq_y_ibuc

        # Create data loaders
        ds = torch.utils.data.TensorDataset(torch.Tensor(np_seq_X_ibuc), torch.Tensor(np_seq_y[inetwork]))
        loader[inetwork] = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True)
            
        np_seq_X[inetwork] = np_seq_X_ibuc
    
    return loader, np_seq_X, np_seq_y


We use the function above and parameters defined in the notebook environment to generate the training, validation, test data loaders

In [ ]:
## For combined bucket network inputs with combined downstream flow data
combined_train_loader, combined_np_train_seq_X, combined_np_train_seq_y = make_combined_data_loader(train_start, train_end, basin_networks_train, combined_network_dictionary, input_combined_vars, output_combined_vars, scaler_in_combined, scaler_out_combined, n_combined_input)
combined_val_loader, combined_np_val_seq_X, combined_np_val_seq_y = make_combined_data_loader(val_start, val_end, basin_networks_val, combined_network_dictionary, input_combined_vars, output_combined_vars, scaler_in_combined, scaler_out_combined, n_combined_input)
combined_test_loader, combined_np_test_seq_X, combined_np_test_seq_y = make_combined_data_loader(test_start, test_end, basin_networks_test, combined_network_dictionary, input_combined_vars, output_combined_vars, scaler_in_combined, scaler_out_combined, n_combined_input)

### 2.5 Training the model: Learning the general response of the example dynamic ''hydrologic" system
Now is the time to train the model! Everything above was done in preparation for this step.

Here we define a function to train the LSTM neural network model with the nn.MSELoss() loss function and using the Adam optimizer and hyperparameters defined above.

**Brief explanation**:
- The training is done for a specified number of epochs.
- For each epoch, the individual characteristics of each bucket and the combined flow of the entire network are processed.
- Data from each basin network are loaded using a PyTorch DataLoader and passed through the LSTM model to predict the individual flow and overflow for each bucket in the network.
- The output is then compared with the target values using the custom loss function. 
- The gradients are calculated and the optimizer is used to update the weights of the model. 
- We use the tqdm library to show the progress of the training. 
- Finally, we estimate the average RMSE for each epoch. 

In [ ]:
def train_model(lstm, train_loader, basin_networks_train, learning_rate, num_epochs):
    criterion = nn.MSELoss()
    criterion = criterion.to(device=device) 
        
    optimizer = optim.Adam(lstm.parameters(), lr = learning_rate[0])
    epoch_bar = tqdm(range(num_epochs), desc = "Training", position = 0, total = num_epochs)

    # Create a dictionary to store the results
    results = {}

    for epoch in epoch_bar:
        for inetwork in basin_networks_train:
            # Initialize results for each network at the beginning
            if inetwork not in results:
                results[inetwork] = {"loss": [], "RMSE": []}

            batch_bar = tqdm(enumerate(train_loader[inetwork]),
                             desc="Network: {}, Epoch: {}".format(str(inetwork), str(epoch)),
                             position = 1, total = len(train_loader[inetwork]), leave = False, disable = True)

            for i, (data, targets) in batch_bar:

                optimizer.zero_grad()

                optimizer = optim.Adam(lstm.parameters(), lr = learning_rate[epoch])

                data = data.to(device = device)
                targets = targets.to(device = device)

                # Forward
                lstm_output = lstm(data)
                loss = criterion(lstm_output, targets)

                #backward
                optimizer.zero_grad()
                loss.backward()

                # gradient descent or adam step
                optimizer.step()

                batch_bar.set_postfix(loss = loss.to(device).item(),
                                      RMSE = "{:.2f}".format(loss ** (1 / 2)),
                                      epoch = epoch)
                batch_bar.update()

            with torch.no_grad():
                rmse_list = []
                for i, (data_, targets_) in enumerate(train_loader[inetwork]):
                    data_ = data_.to(device = device)
                    targets_ = targets_.to(device = device)

                    lstm_output_ = lstm(data_)
                    MSE_ = criterion(lstm_output_, targets_)
                    rmse_list.append(MSE_ ** (1 / 2))

            #meanrmse = np.mean(np.array(torch.Tensor(rmse_list)))
            meanrmse = torch.stack(rmse_list).mean().item()
            epoch_bar.set_postfix(loss = loss.cpu().item(),
                                  RMSE = "{:.2f}".format(meanrmse),
                                  epoch = epoch)

            results[inetwork]["loss"].append(loss.cpu().item())
            results[inetwork]["RMSE"].append(meanrmse)

            batch_bar.update()

        #clear_output()

    return lstm, results


Run the train model functon by prescribing the buckets and data to use for training and the intantiated LSTM

In [ ]:
## For combined bucket network inputs with combined downstream flow data
lstm_combined, results_combined = train_model(lstm_combined, combined_train_loader, basin_networks_train, learning_rate_combined, num_epochs_combined)

### 2.5 Visualizing the learning curves

 We plot the loss and root mean square error (RMSE) metrics for each epoch to check if the model fitting has converged

In [ ]:
def viz_learning_curve(results):
    fig = plt.figure(figsize=(12, 3))
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)
    for inetwork in basin_networks_train:
        num_epochs_executed = len(results[inetwork]['loss'])
        ax1.plot(range(num_epochs_executed), results[inetwork]['loss'])
        ax2.plot(range(num_epochs_executed), results[inetwork]['RMSE'])
    ax1.set_ylabel('Loss')
    ax2.set_ylabel('RMSE')
    ax1.set_xlabel('Epoch')
    ax2.set_xlabel('Epoch')
    plt.suptitle("Learning curves for each network") 
    plt.show()


In [ ]:
## For combined bucket network inputs with combined downstream flow data
viz_learning_curve(results_combined)

### 2.6 Checking that the model works on the validation data
Now that we have a trained model, we can see how it works on our validation split.

In [ ]:
def print_model_metrics(metrics_list, title):
    metrics_df = pd.DataFrame(metrics_list).mean()
    print(f"*** Model Metrics - {title} ***")
    print("Average NSE Spigot: ", metrics_df['spigot_nse'])
    print("Average NSE Overflow: ", metrics_df['overflow_nse'])
    print("Average RMSE Spigot: ", metrics_df['spigot_rmse'])
    print("Average RMSE Overflow: ", metrics_df['overflow_rmse'])
    print("Average MAE Spigot: ", metrics_df['spigot_mae'])
    print("Average MAE Overflow: ", metrics_df['overflow_mae'])

In [ ]:
## For combined bucket network inputs with combined downstream flow data
model_combined_metrics_list = []
for inetwork in basin_networks_val:
    network_metrics_df = check_validation_period_combined(lstm_combined, combined_np_val_seq_X, combined_network_dictionary, inetwork, n_plot=100, verbose=False)
    model_combined_metrics_list.append(network_metrics_df.iloc[0].to_dict())

In [ ]:
## For combined bucket network inputs with combined downstream flow data
print_model_metrics(model_combined_metrics_list, "Validation: Combined bucket network inputs with combined downstream flow data")

# 3. Final model evaluation

Finally, we evaluate the model on the test data to see how well it performs on unseen data.

In [ ]:
def check_test_period_combined(lstm_combined, combined_np_test_seq_X, inetwork, n_plot=100, verbose=False):
    def __make_prediction(df, ibuc):
        lstm_output_test = lstm_combined(torch.Tensor(combined_np_test_seq_X[inetwork]).to(device=device))
        test_spigot_prediction = []
        test_overflow_prediction = []

        for i in range(lstm_output_test.shape[0]):
            spigot_pred = (lstm_output_test[i, -1, (ibuc * 2) + 1].cpu().detach().numpy() * np.std(df.loc[train_start:train_end, 'q_spigot_b' + str(ibuc)])) + np.mean(df.loc[train_start:train_end, 'q_spigot_b' + str(ibuc)])
            overflow_pred = (lstm_output_test[i, -1, (ibuc * 2)].cpu().detach().numpy() * np.std(df.loc[train_start:train_end, 'q_overflow_b' + str(ibuc)])) + np.mean(df.loc[train_start:train_end, 'q_overflow_b' + str(ibuc)])

            spigot_pred = max(spigot_pred, 0)
            overflow_pred = max(overflow_pred, 0)

            test_spigot_prediction.append(spigot_pred)
            test_overflow_prediction.append(overflow_pred)

        return test_spigot_prediction, test_overflow_prediction

    def __compute_metrics_combined(df, test_spigot_prediction, test_overflow_prediction, ibuc):
        # Use specific columns for each bucket
        spigot_out = df.loc[test_start:test_end, 'q_spigot_b' + str(ibuc)]
        spigot_mean = np.mean(spigot_out)
        spigot_pred_variance = 0
        spigot_obs_variance = 0
        spigot_abs_diff = 0

        overflow_out = df.loc[test_start:test_end, 'q_overflow_b' + str(ibuc)]
        overflow_mean = np.mean(overflow_out)
        overflow_pred_variance = 0
        overflow_obs_variance = 0
        overflow_abs_diff = 0

        for i, pred_spigot in enumerate(test_spigot_prediction):
            t = i + seq_length - 1
            spigot_pred_variance += np.power((pred_spigot - spigot_out.iloc[t]), 2)
            spigot_obs_variance += np.power((spigot_mean - spigot_out.iloc[t]), 2)
            spigot_abs_diff += np.abs(pred_spigot - spigot_out.iloc[t])

        for i, pred_overflow in enumerate(test_overflow_prediction):
            t = i + seq_length - 1
            overflow_pred_variance += np.power((pred_overflow - overflow_out.iloc[t]), 2)
            overflow_obs_variance += np.power((overflow_mean - overflow_out.iloc[t]), 2)            
            overflow_abs_diff += np.abs(pred_overflow - overflow_out.iloc[t])

        spigot_nse = 1 - (spigot_pred_variance / spigot_obs_variance)
        overflow_nse = 1 - (overflow_pred_variance / overflow_obs_variance)
        
        spigot_rmse = np.sqrt(spigot_pred_variance / len(test_spigot_prediction))
        overflow_rmse = np.sqrt(overflow_pred_variance / len(test_overflow_prediction))
        
        spigot_mae = spigot_abs_diff / len(test_spigot_prediction)
        overflow_mae = overflow_abs_diff / len(test_overflow_prediction)

        return spigot_nse, overflow_nse, spigot_rmse, overflow_rmse, spigot_mae, overflow_mae

    def __compute_mass_balance_combined(df, ibuc):
        suffix = '_b' + str(ibuc)  # Suffix to identify the specific bucket
        mass_in = df.sum()['precip' + suffix]
        mass_out = df.sum()['et' + suffix] + df.sum()['q_overflow' + suffix] + df.sum()['q_spigot' + suffix] + df['h_bucket' + suffix].iloc[-1]
        return mass_in, mass_out

    if verbose: print("*** Network", inetwork, "***")

    df = retrieve_combined_data(inetwork, combined_network_dictionary)  # Retrieves the combined DataFrame for this network
    metrics_list = []

    for ibuc in range(n_buckets_per_network):
        # Prediction is made with the combined DataFrame, but specifying the bucket
        test_spigot_prediction, test_overflow_prediction = __make_prediction(df, ibuc)
        # Compute the metrics for the predictions of this bucket
        spigot_nse, overflow_nse, spigot_rmse, overflow_rmse, spigot_mae, overflow_mae = __compute_metrics_combined(df, test_spigot_prediction, test_overflow_prediction, ibuc)
        mass_in, mass_out = __compute_mass_balance_combined(df, ibuc)

        new_row = {
            'spigot_nse': spigot_nse, 'overflow_nse': overflow_nse, 
            'spigot_rmse': spigot_rmse, 'overflow_rmse': overflow_rmse, 
            'spigot_mae': spigot_mae, 'overflow_mae': overflow_mae,
            'mass_in': mass_in, 'mass_out': mass_out
        }
        metrics_list.append(new_row)

        if verbose:
            print("Bucket", ibuc)
            print("NSE Spigot: ", spigot_nse)
            print("NSE Overflow: ", overflow_nse)
            print("RMSE Spigot: ", spigot_rmse)
            print("RMSE Overflow: ", overflow_rmse)
            print("MAE Spigot: ", spigot_mae)
            print("MAE Overflow: ", overflow_mae)
            print("Mass into the system: ", mass_in)
            print("Mass out or left over:", mass_out)
            print("Percent mass residual: {:.0%}".format((mass_in - mass_out) / mass_in))

            # Plot the predictions for this bucket
            fig = plt.figure(figsize=(12, 3))
            ax1 = fig.add_subplot(1, 2, 1)
            ax2 = fig.add_subplot(1, 2, 2)
            ax1.plot(df.loc[test_start + seq_length - 1:test_start + n_plot + seq_length - 1, 'q_spigot_b' + str(ibuc)].values, label = "Spigot out")
            ax1.plot(test_spigot_prediction[:n_plot], label="LSTM spigot out")
            ax1.legend(loc="upper right")
            ax1.set_xlabel('Time (hours)')
            ax1.set_ylabel('Flow rate (m$^3$/s)')
            ax2.plot(df.loc[test_start + seq_length - 1:test_start + n_plot + seq_length - 1, 'q_overflow_b' + str(ibuc)].values, label = "Overflow")
            ax2.plot(test_overflow_prediction[:n_plot], label="LSTM Overflow")
            ax2.legend(loc="upper right")
            ax2.set_xlabel('Time (hours)')
            ax2.set_ylabel('Flow rate (m$^3$/s)')
            plt.show()
            plt.close()

    metrics_df = pd.DataFrame(metrics_list)
    avg_metrics = metrics_df[['spigot_nse', 'overflow_nse', 'spigot_rmse', 'overflow_rmse', 'spigot_mae', 'overflow_mae']].mean()
    total_mass_in = metrics_df['mass_in'].sum()
    total_mass_out = metrics_df['mass_out'].sum()
    total_mass_residual_percent = (total_mass_in - total_mass_out) / total_mass_in

    network_metrics_df = pd.DataFrame([avg_metrics])
    network_metrics_df['total_mass_in'] = total_mass_in
    network_metrics_df['total_mass_out'] = total_mass_out
    network_metrics_df['total_mass_residual_percent'] = total_mass_residual_percent

    if verbose:
        print("*** Network Metrics ***")
        print("Network", inetwork)
        print("NSE Spigot: ", network_metrics_df['spigot_nse'].iloc[0])
        print("NSE Overflow: ", network_metrics_df['overflow_nse'].iloc[0])
        print("RMSE Spigot: ", network_metrics_df['spigot_rmse'].iloc[0])
        print("RMSE Overflow: ", network_metrics_df['overflow_rmse'].iloc[0])
        print("MAE Spigot: ", network_metrics_df['spigot_mae'].iloc[0])
        print("MAE Overflow: ", network_metrics_df['overflow_mae'].iloc[0])
        print("Mass into the system: ", network_metrics_df['total_mass_in'].iloc[0])
        print("Mass out or left over: ", network_metrics_df['total_mass_out'].iloc[0])
        print("Percent mass residual: {:.0%}".format(network_metrics_df['total_mass_residual_percent'].iloc[0]))

    return network_metrics_df


In [ ]:
## For combined bucket network inputs with combined downstream flow data
test_model_combined_metrics_list = []
for inetwork in basin_networks_test:
    network_metrics_df = check_test_period_combined(lstm_combined, combined_np_test_seq_X, inetwork, n_plot=100, verbose=False)
    test_model_combined_metrics_list.append(network_metrics_df.iloc[0].to_dict())

In [ ]:
## For combined bucket network inputs with combined downstream flow data
print_model_metrics(test_model_combined_metrics_list, "Testing: Combined bucket network inputs with combined downstream flow data")

In [ ]:
print("End")